In [2]:
#   |'''''''''''''╔╬╬╬╬╬╬╬╬   _____  _____      _____  _____      ___   __
#   |            ╔╬╬╬╬╬╬╬╬╬  |\   _ \  _  \    |\   _ \  _  \    |\  \|\  \
#   | ░░         ╬╬╬╬╬╬╬╬╬╬  \ \  \\__\ \  \   \ \  \\__\ \  \   \ \  \/  /|_
#    ░░░░        ╬╬╬╬╬╬╬╬╬╬   \ \  \|__| \  \   \ \  \|__| \  \   \ \   ___  \
#   ░░░░░╦╬╦    ╔╬╬╬╬╬╬╬╬╬╬    \ \  \   \ \  \   \ \  \   \ \  \   \ \  \ \   \
#  ░░░░░╬╬╬╬ ▓▓└╬╬╬╬╬╬╬╬╬╬╬     \ \__\   \ \__\   \ \__\   \ \__\   \ \__\ \___\
# ░░░░░╔╬╬╬ ▓▓▓  ╓╬╬╬╬╬╬╬╬╬      \|__|    \|__|    \|__|    \|__|    \|__| \|__|
# ░░░░░╠╬╬╬ ▓▓▓  └╬╬╬╬╬╬╬╬╬
#  ░░░░└╬╬╬╬ ▓▓   ╬╬╬╬╬╬╬╬╬  Lehrstuhl für Mensch-Maschine-Kommunikation
#  ░░░░░╙╬╬╬╩            ╬╬  Technische Universität München
#   ░░░░░░╚ '''''''''''''''  Author: Tobias Watzel
#    ░░░                     Copyright 2020
#

%matplotlib widget
import cv2
import glob
import re
import io
import ipywidgets as widgets
import numpy as np
from IPython.display import clear_output, Image, display
import PIL.Image
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib as mpl
from matplotlib import cm
from functools import partial

# define figure
fig = None

## Versuchsbeschreibung
Wählen Sie ein Bild und eine Konvertierungsmethode aus. Es wird eine Histogrammangleichung vorgenommen. Welche Auswirkung hat die Histogrammangleichung auf das Bild? 

In [5]:
if fig:
    plt.close(fig)

fig = plt.figure(figsize = (7, 4))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
# create subaxes
left, bottom, width, height = ax1.get_position().bounds
ax1_subaxes = fig.add_axes([left, bottom - height * 0.05 - 0.01, width, height * 0.05])
left, bottom, width, height = ax2.get_position().bounds
ax2_subaxes = fig.add_axes([left, bottom - height * 0.05 - 0.01, width, height * 0.05])
norm = mpl.colors.Normalize(vmin = 0, vmax = 255)
cb1 = mpl.colorbar.ColorbarBase(ax1_subaxes, norm = norm, orientation = 'horizontal', cmap = cm.Greys_r)
cb1 = mpl.colorbar.ColorbarBase(ax2_subaxes, norm = norm, orientation = 'horizontal', cmap = cm.Greys_r)
ax1.get_xaxis().set_visible(False)
ax2.get_xaxis().set_visible(False)
#hist2 = ax2.hist(np.zeros(1))

# create dict with name of image and its path (FIX? hacked...)
dict_images = {entry.replace('/',' ').replace('.',' ').split()[1]:entry for entry in sorted(glob.glob("images/*"))}

def histogram_adjustment(histogram_in):
    pass

def convert_image(a, fmt='png'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    return f.getvalue()

def rgb_to_gray_colorimetric(image_in):
    # check, if RGB
    assert image_in.shape[2] == 3
    return np.uint8(0.3 * image_in[:, :, 0] + 0.59 * image_in[:, :, 1] + 0.11 * image_in[:, :, 2])

def rgb_to_gray_average(image_in):
    # check, if RGB
    assert image_in.shape[2] == 3
    return np.uint8(np.mean(image_in, axis=2))

def display_images(image_name, conversion = 'technisch'):  
    # read image
    image = cv2.imread(dict_images[image_name])
    
    # image in RGB (just switching dims)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # transform images
    image_gray = None
    if conversion == 'technisch':
        image_gray = rgb_to_gray_average(image_rgb)
    else:
        image_gray = rgb_to_gray_colorimetric(image_rgb)
    
    # plot histgrams
    ax1.cla()
    ax2.cla()
    ax1.set_title('Histogram (%s)' % conversion)
    ax2.set_title('Histogram nach Angleichung')
    
    image_equalized = cv2.equalizeHist(image_gray)
    hist, bins  = np.histogram(image_equalized.flatten(), 256, [0,256])

    ax1.hist(image_gray.ravel(), 256, [0, 256])
    ax2.hist(image_equalized.ravel(), 256, [0, 256])
    
    ax1.set_xlim(0, 255)
    ax2.set_xlim(0, 255)
    
    image_1.value = convert_image(image_rgb)
    image_2.value = convert_image(image_gray)
    image_3.value = convert_image(image_equalized)
    
    label_2.value = 'Bild grau (%s)' % conversion
    label_3.value = 'Bild nach Angleichung (%s)' % conversion
    
def change_val(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] in list(dict_images.keys()):
            display_images(change['new'], select_conversion.value)
        else:
            display_images(select.value, change['new'])

        
select = widgets.Select(
    options=list(dict_images.keys()),
    value='birne',
    description='Dateiliste:')

select_conversion = widgets.Select(
    options=['technisch', 'wahrnehmungsgerecht'],
    value='technisch',
    description='Dateiliste:')

select_box = widgets.HBox([select, select_conversion])

file = open("images/birne.png", "rb")
image_test = file.read()

image_1 = widgets.Image(value=image_test, format='png', width=300, height=400)
image_2 = widgets.Image(value=image_test, format='png', width=300, height=400)
image_3 = widgets.Image(value=image_test, format='png', width=300, height=400)
label_1 = widgets.Label(value='Ursprüngliches Bild')
label_2 = widgets.Label(value='Bild grau (%s)' % select_conversion.value)
label_3 = widgets.Label(value='Bild nach Angleichung (%s)' % select_conversion.value)

box_1 = widgets.VBox([label_1, image_1])
box_2 = widgets.VBox([label_2, image_2])
box_3 = widgets.VBox([label_3, image_3])

items = [box_1, box_2, box_3]
box = widgets.HBox(items)

display(select_box)
display_images('birne')
display(box)

#select.observe(change_val)
select.observe(change_val)
select_conversion.observe(change_val)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Auswirkung
## 1. Verbesserung des Kontrasts: Optimale Verteilung der Helligkeitsstufen für ausgewogenere Kontrastverhältnis.

## 2. Erhöhung von Detailgenauigkeit: Bessere Hervorhebung feiner Details durch effektivere Nutzung des Helligkeitsbereichs.

## 3. Anpassung der Farbbalance: Beeinflussung der Farbintensitäten für eine ausgeglichene Farbwiedergabe.
